In [5]:
import pandas as pd
from sklearn.model_selection import train_test_split 
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score,precision_score, recall_score, confusion_matrix
file_path = 'url_dataset.csv'
data = pd.read_csv(file_path)

In [2]:
data.info(), data.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 18 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Domain         10000 non-null  object
 1   Have_IP        10000 non-null  int64 
 2   Have_At        10000 non-null  int64 
 3   URL_Length     10000 non-null  int64 
 4   URL_Depth      10000 non-null  int64 
 5   Redirection    10000 non-null  int64 
 6   https_Domain   10000 non-null  int64 
 7   TinyURL        10000 non-null  int64 
 8   Prefix/Suffix  10000 non-null  int64 
 9   DNS_Record     10000 non-null  int64 
 10  Web_Traffic    10000 non-null  int64 
 11  Domain_Age     10000 non-null  int64 
 12  Domain_End     10000 non-null  int64 
 13  iFrame         10000 non-null  int64 
 14  Mouse_Over     10000 non-null  int64 
 15  Right_Click    10000 non-null  int64 
 16  Web_Forwards   10000 non-null  int64 
 17  Label          10000 non-null  int64 
dtypes: int64(17), object(1)
mem

(None,
             Have_IP       Have_At    URL_Length     URL_Depth   Redirection  \
 count  10000.000000  10000.000000  10000.000000  10000.000000  10000.000000   
 mean       0.005500      0.022600      0.773400      3.072000      0.013500   
 std        0.073961      0.148632      0.418653      2.128631      0.115408   
 min        0.000000      0.000000      0.000000      0.000000      0.000000   
 25%        0.000000      0.000000      1.000000      2.000000      0.000000   
 50%        0.000000      0.000000      1.000000      3.000000      0.000000   
 75%        0.000000      0.000000      1.000000      4.000000      0.000000   
 max        1.000000      1.000000      1.000000     20.000000      1.000000   
 
        https_Domain       TinyURL  Prefix/Suffix    DNS_Record   Web_Traffic  \
 count  10000.000000  10000.000000   10000.000000  10000.000000  10000.000000   
 mean       0.000200      0.090300       0.093200      0.100800      0.845700   
 std        0.014141      0.

In [3]:
outliers = {}
for column in data.select_dtypes(include=['int64']).columns:
    Q1 = data[column].quantile(0.25)
    Q3 = data[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    outlier_count = ((data[column] < lower_bound) | (data[column] > upper_bound)).sum()
    outliers[column] = outlier_count

outliers

{'Have_IP': 55,
 'Have_At': 226,
 'URL_Length': 2266,
 'URL_Depth': 357,
 'Redirection': 135,
 'https_Domain': 2,
 'TinyURL': 903,
 'Prefix/Suffix': 932,
 'DNS_Record': 1008,
 'Web_Traffic': 1543,
 'Domain_Age': 0,
 'Domain_End': 1901,
 'iFrame': 909,
 'Mouse_Over': 666,
 'Right_Click': 7,
 'Web_Forwards': 1053,
 'Label': 0}

In [ ]:
X = data.drop("Label", axis=1)
y = data["Label"]
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=.25)

,Domain,Have_IP,Have_At,URL_Length,URL_Depth,Redirection,https_Domain,TinyURL,Prefix/Suffix,DNS_Record,Web_Traffic,Domain_Age,Domain_End,iFrame,Mouse_Over,Right_Click,Web_Forwards
8549,fixertawa.blogspot.com,0,0,0,0,0,0,1,0,1,1,1,1,0,0,1,0
6638,muggetlogin.oa.r.appspot.com,0,0,0,1,0,0,1,0,1,1,1,1,0,0,1,0
2951,steamcommunity.com,0,0,1,5,0,0,0,0,0,1,0,1,0,0,1,0
4535,askubuntu.com,0,0,1,3,0,0,0,0,0,1,0,1,0,0,1,0
7866,evoadministrador.com,0,0,0,2,0,0,0,0,0,1,1,1,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6250,viccats.camcam.dns-cloud.net,0,0,1,3,0,0,0,1,0,0,0,0,0,0,1,0
7552,agro-hotel.com,0,0,0,1,0,0,0,1,0,1,0,1,0,0,1,0
3652,kotaku.com,0,0,1,1,0,0,0,0,0,1,1,1,0,0,1,0
1704,mirtesen.ru,0,0,1,1,0,0,0,0,0,1,0,1,0,0,1,0


In [ ]:
#Neural Network 

In [ ]:
#Random Forest